In [4]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import random

from activation import *
from loss import *
from linear import * 

In [5]:

# Génération des données

# Création des modules
linear1 = Linear(2, 3)  # Couche linéaire avec 3 neurones
tanh = TanH()           # Activation TanH
linear2 = Linear(3, 1)   # Couche linéaire avec 1 neurone (pour la sortie binaire)
sigmoid = Sigmoid()      # Activation Sigmoide

# Définition de la fonction de perte
loss_fn = MSELoss()

